# Finance Agent using LangChain (LlaMa-3 | Groq API)

## Table of Contents

1. **Load LLM (LlaMa-3)**
2. **Define Finance Tools**
3. **Create Finance Agent (LLM + Tools)**

## Finance Agent Capabilities

* **Stock Company Basic Information**
    * Address, Industry, Sector, business summary, website, etc.
    * Top Company Officers
* **Stock Financials**
    * Market Cap, current price, ebitda, total debt, total revenue, debt-to-equity, P/E ratio, operating margins, ebitda margins, operating cashflow, free cashflow, etc.
* **Stock Dividend & Earnings Dates**
* **Information about Mutual Fund Holders**
* **Information about Institutional Holders**
* **Stock Splits History**
* **Stock Ratings Upgrades & Downgrades by various Companies**
* **News Articles about Stock**

In [1]:
import dotenv

dotenv.load_dotenv(dotenv.find_dotenv())

True

In [2]:
import os

groq_api_key = os.environ["GROQ_API_KEY"]

In [3]:
import yfinance as yf

yf.__version__

'0.2.40'

## 1. Load LLM

* Login to **https://console.groq.com** and create API Key.

### Groq Models

ID|	REQUESTS PER MINUTE|	REQUESTS PER DAY|	TOKENS PER MINUTE
-|-|-|-
llama3-70b-8192	|30	|14,400	|6,000
llama3-8b-8192	|30	|14,400	|30,000
gemma-7b-it	|30	|14,400	|15,000
mixtral-8x7b-32768	|30	|14,400	|5,000


* **pip install langchain-groq**

In [5]:
from langchain_groq import ChatGroq

llama3 = ChatGroq(api_key=groq_api_key, model="llama3-8b-8192")

llama3

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7f5a6b94d420>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7f5a6b94c760>, model_name='llama3-8b-8192', groq_api_key=SecretStr('**********'))

In [6]:
llama3.invoke("Hello, How are you?")

AIMessage(content="Hello! I'm just a language model, so I don't have feelings or emotions like humans do, but I'm functioning properly and ready to help with any questions or tasks you may have! How about you? How's your day going so far?", response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 16, 'total_tokens': 68, 'completion_time': 0.040477154, 'prompt_time': 0.004461602, 'queue_time': None, 'total_time': 0.044938756}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_873a560973', 'finish_reason': 'stop', 'logprobs': None}, id='run-e9e980fa-513f-49dc-bfc4-d661bff91293-0', usage_metadata={'input_tokens': 16, 'output_tokens': 52, 'total_tokens': 68})

## 2. Define Tools

In [17]:
from langchain_core.tools import tool, StructuredTool
from datetime import date

@tool
def company_information(ticker: str) -> dict:
    """Use this tool to retrieve company information like address, industry, sector, company officers, business summary, website,
       marketCap, current price, ebitda, total debt, total revenue, debt-to-equity, etc."""
    
    ticker_obj = yf.Ticker(ticker)
    ticker_info = ticker_obj.get_info()

    return ticker_info

@tool
def last_dividend_and_earnings_date(ticker: str) -> dict:
    """
    Use this tool to retrieve company's last dividend date and earnings release dates.
    It does not provide information about historical dividend yields.
    """
    ticker_obj = yf.Ticker(ticker)
    
    return ticker_obj.get_calendar()

@tool
def summary_of_mutual_fund_holders(ticker: str) -> dict:
    """
    Use this tool to retrieve company's top mutual fund holders. 
    It also returns their percentage of share, stock count and value of holdings.
    """
    ticker_obj = yf.Ticker(ticker)
    mf_holders = ticker_obj.get_mutualfund_holders()
    
    return mf_holders.to_dict(orient="records")

@tool
def summary_of_institutional_holders(ticker: str) -> dict:
    """
    Use this tool to retrieve company's top institutional holders. 
    It also returns their percentage of share, stock count and value of holdings.
    """
    ticker_obj = yf.Ticker(ticker)   
    inst_holders = ticker_obj.get_institutional_holders()
    
    return inst_holders.to_dict(orient="records")

@tool
def stock_grade_updrages_downgrades(ticker: str) -> dict:
    """
    Use this to retrieve grade ratings upgrades and downgrades details of particular stock.
    It'll provide name of firms along with 'To Grade' and 'From Grade' details. Grade date is also provided.
    """
    ticker_obj = yf.Ticker(ticker)
    
    curr_year = date.today().year
    
    upgrades_downgrades = ticker_obj.get_upgrades_downgrades()
    upgrades_downgrades = upgrades_downgrades.loc[upgrades_downgrades.index > f"{curr_year}-01-01"]
    upgrades_downgrades = upgrades_downgrades[upgrades_downgrades["Action"].isin(["up", "down"])]
    
    return upgrades_downgrades.to_dict(orient="records")

@tool
def stock_splits_history(ticker: str) -> dict:
    """
    Use this tool to retrieve company's historical stock splits data.
    """
    ticker_obj = yf.Ticker(ticker)
    hist_splits = ticker_obj.get_splits()
    
    return hist_splits.to_dict()

@tool
def stock_news(ticker: str) -> dict:
    """
    Use this to retrieve latest news articles discussing particular stock ticker.
    """
    ticker_obj = yf.Ticker(ticker)
    
    return ticker_obj.get_news()


#stock_news_tool = StructuredTool.from_function(stock_news)

## 3.  Define Agent (LLM + Tools)

In [18]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder

tools = [
         company_information,
         last_dividend_and_earnings_date,
         stock_splits_history,
         summary_of_mutual_fund_holders,
         summary_of_institutional_holders, 
         stock_grade_updrages_downgrades,
         stock_news
]

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Try to answer user query using available tools.",
        ),
        MessagesPlaceholder(variable_name="messages"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

llama3 = ChatGroq(api_key=groq_api_key, model="llama3-8b-8192", temperature=0)

finance_agent = create_tool_calling_agent(llama3, tools, prompt)

finance_agent_executor = AgentExecutor(agent=finance_agent, tools=tools, verbose=True)

In [19]:
from langchain_core.messages import HumanMessage

resp = finance_agent_executor.invoke({"messages": [HumanMessage(content="What is address of microsoft?")]})



> Entering new AgentExecutor chain...

Invoking: `company_information` with `{'ticker': 'MSFT'}`


{'address1': 'One Microsoft Way', 'city': 'Redmond', 'state': 'WA', 'zip': '98052-6399', 'country': 'United States', 'phone': '425 882 8080', 'website': 'https://www.microsoft.com', 'industry': 'Software - Infrastructure', 'industryKey': 'software-infrastructure', 'industryDisp': 'Software - Infrastructure', 'sector': 'Technology', 'sectorKey': 'technology', 'sectorDisp': 'Technology', 'longBusinessSummary': 'Microsoft Corporation develops and supports software, services, devices and solutions worldwide. The Productivity and Business Processes segment offers office, exchange, SharePoint, Microsoft Teams, office 365 Security and Compliance, Microsoft viva, and Microsoft 365 copilot; and office consumer services, such as Microsoft 365 consumer subscriptions, Office licensed on-premises, and other office services. This segment also provides LinkedIn; and dynamics business solutions, includ

In [20]:
print(resp["output"])

The address of Microsoft is One Microsoft Way, Redmond, WA 98052-6399, United States.


In [21]:
#finance_agent_executor.verbose = False

In [22]:
resp = finance_agent_executor.invoke({"messages": [HumanMessage(content="What is last close price of microsoft?")]})



> Entering new AgentExecutor chain...

Invoking: `last_dividend_and_earnings_date` with `{'ticker': 'MSFT'}`


{'Dividend Date': datetime.date(2024, 9, 11), 'Ex-Dividend Date': datetime.date(2024, 8, 14), 'Earnings Date': [datetime.date(2024, 7, 23), datetime.date(2024, 7, 29)], 'Earnings High': 3.31, 'Earnings Low': 2.86, 'Earnings Average': 2.93, 'Revenue High': 65543000000, 'Revenue Low': 64000000000, 'Revenue Average': 64348000000}
Invoking: `company_information` with `{'ticker': 'MSFT'}`


{'address1': 'One Microsoft Way', 'city': 'Redmond', 'state': 'WA', 'zip': '98052-6399', 'country': 'United States', 'phone': '425 882 8080', 'website': 'https://www.microsoft.com', 'industry': 'Software - Infrastructure', 'industryKey': 'software-infrastructure', 'industryDisp': 'Software - Infrastructure', 'sector': 'Technology', 'sectorKey': 'technology', 'sectorDisp': 'Technology', 'longBusinessSummary': 'Microsoft Corporation develops and supports software, services, devices and solutions

In [23]:
print(resp["output"])

The last close price of Microsoft (MSFT) is 467.56.


In [24]:
resp = finance_agent_executor.invoke({"messages": [HumanMessage(content="What is dividend yield for microsoft stock?")]})



> Entering new AgentExecutor chain...

Invoking: `company_information` with `{'ticker': 'MSFT'}`


{'address1': 'One Microsoft Way', 'city': 'Redmond', 'state': 'WA', 'zip': '98052-6399', 'country': 'United States', 'phone': '425 882 8080', 'website': 'https://www.microsoft.com', 'industry': 'Software - Infrastructure', 'industryKey': 'software-infrastructure', 'industryDisp': 'Software - Infrastructure', 'sector': 'Technology', 'sectorKey': 'technology', 'sectorDisp': 'Technology', 'longBusinessSummary': 'Microsoft Corporation develops and supports software, services, devices and solutions worldwide. The Productivity and Business Processes segment offers office, exchange, SharePoint, Microsoft Teams, office 365 Security and Compliance, Microsoft viva, and Microsoft 365 copilot; and office consumer services, such as Microsoft 365 consumer subscriptions, Office licensed on-premises, and other office services. This segment also provides LinkedIn; and dynamics business solutions, includ

{'Dividend Date': datetime.date(2024, 9, 11), 'Ex-Dividend Date': datetime.date(2024, 8, 14), 'Earnings Date': [datetime.date(2024, 7, 23), datetime.date(2024, 7, 29)], 'Earnings High': 3.31, 'Earnings Low': 2.86, 'Earnings Average': 2.93, 'Revenue High': 65543000000, 'Revenue Low': 64000000000, 'Revenue Average': 64348000000}The dividend yield for Microsoft stock is 0.0064.

> Finished chain.


In [25]:
print(resp["output"])

The dividend yield for Microsoft stock is 0.0064.


In [26]:
resp = finance_agent_executor.invoke({"messages": [HumanMessage(content="What are ebitda, total debt, total revenue and debt-to-equity for Nvidia stock?")]})



> Entering new AgentExecutor chain...

Invoking: `company_information` with `{'ticker': 'NVDA'}`


{'address1': '2788 San Tomas Expressway', 'city': 'Santa Clara', 'state': 'CA', 'zip': '95051', 'country': 'United States', 'phone': '408 486 2000', 'website': 'https://www.nvidia.com', 'industry': 'Semiconductors', 'industryKey': 'semiconductors', 'industryDisp': 'Semiconductors', 'sector': 'Technology', 'sectorKey': 'technology', 'sectorDisp': 'Technology', 'longBusinessSummary': "NVIDIA Corporation provides graphics and compute and networking solutions in the United States, Taiwan, China, Hong Kong, and internationally. The Graphics segment offers GeForce GPUs for gaming and PCs, the GeForce NOW game streaming service and related infrastructure, and solutions for gaming platforms; Quadro/NVIDIA RTX GPUs for enterprise workstation graphics; virtual GPU or vGPU software for cloud-based visual and virtual computing; automotive platforms for infotainment systems; and Omniverse software f

In [27]:
print(resp["output"])

Based on the provided information, here are the values for ebitda, total debt, total revenue, and debt-to-equity for Nvidia stock:

* EBITDA: 49,274,998,784
* Total Debt: 11,237,000,192
* Total Revenue: 79,773,991,040
* Debt-to-Equity: 22.866

Please note that these values are based on the provided data and may not reflect the current or up-to-date information.


In [28]:
resp = finance_agent_executor.invoke({"messages": [HumanMessage(content="Summarize Key people at apple")]})



> Entering new AgentExecutor chain...

Invoking: `company_information` with `{'ticker': 'AAPL'}`


{'address1': 'One Apple Park Way', 'city': 'Cupertino', 'state': 'CA', 'zip': '95014', 'country': 'United States', 'phone': '408 996 1010', 'website': 'https://www.apple.com', 'industry': 'Consumer Electronics', 'industryKey': 'consumer-electronics', 'industryDisp': 'Consumer Electronics', 'sector': 'Technology', 'sectorKey': 'technology', 'sectorDisp': 'Technology', 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover an

In [29]:
print(resp["output"])

Based on the tool call output, here is a summary of the key people at Apple:

* CEO: Tim Cook
* CFO: Luca Maestri
* Chief Operating Officer: Jeffrey E. Williams
* Senior VP, General Counsel & Secretary: Katherine L. Adams
* Senior Vice President of Retail: Deirdre O'Brien
* Chief Technology Officer: James Wilson
* Director of Investor Relations: Suhasini Chandramouli
* Senior Vice President of Worldwide Marketing: Greg Joswiak
* Head of Corporate Development: Adrian Perica

Note that this information is based on the company's latest available data and may not reflect the current leadership team.


In [32]:
resp = finance_agent_executor.invoke({"messages": [HumanMessage(content="What was last dividend date for apple stock?")]})



> Entering new AgentExecutor chain...

Invoking: `last_dividend_and_earnings_date` with `{'ticker': 'AAPL'}`


{'Dividend Date': datetime.date(2024, 5, 15), 'Ex-Dividend Date': datetime.date(2024, 5, 9), 'Earnings Date': [datetime.date(2024, 8, 1)], 'Earnings High': 1.39, 'Earnings Low': 1.27, 'Earnings Average': 1.33, 'Revenue High': 86676000000, 'Revenue Low': 82022600000, 'Revenue Average': 83859400000}
Invoking: `last_dividend_and_earnings_date` with `{'ticker': 'AAPL'}`


{'Dividend Date': datetime.date(2024, 5, 15), 'Ex-Dividend Date': datetime.date(2024, 5, 9), 'Earnings Date': [datetime.date(2024, 8, 1)], 'Earnings High': 1.39, 'Earnings Low': 1.27, 'Earnings Average': 1.33, 'Revenue High': 86676000000, 'Revenue Low': 82022600000, 'Revenue Average': 83859400000}The last dividend date for Apple stock is May 15, 2024.

> Finished chain.


In [33]:
print(resp["output"])

The last dividend date for Apple stock is May 15, 2024.


In [35]:
resp = finance_agent_executor.invoke({"messages": [HumanMessage(content="Top mutual fund holders of microsoft stock.")]})



> Entering new AgentExecutor chain...

Invoking: `summary_of_mutual_fund_holders` with `{'ticker': 'MSFT'}`


[{'Date Reported': Timestamp('2023-06-30 00:00:00'), 'Holder': 'Vanguard Total Stock Market Index Fund', 'pctHeld': 0.0312, 'Shares': 231883087, 'Value': 78965466446}, {'Date Reported': Timestamp('2023-06-30 00:00:00'), 'Holder': 'Vanguard 500 Index Fund', 'pctHeld': 0.0238, 'Shares': 177034936, 'Value': 60287477105}, {'Date Reported': Timestamp('2023-08-31 00:00:00'), 'Holder': 'Fidelity 500 Index Fund', 'pctHeld': 0.0115, 'Shares': 85181943, 'Value': 27919233637}, {'Date Reported': Timestamp('2023-09-30 00:00:00'), 'Holder': 'SPDR S&P 500 ETF Trust', 'pctHeld': 0.0112, 'Shares': 83514381, 'Value': 26369665800}, {'Date Reported': Timestamp('2023-09-30 00:00:00'), 'Holder': 'iShares Core S&P 500 ETF', 'pctHeld': 0.0095, 'Shares': 70264910, 'Value': 22186145332}, {'Date Reported': Timestamp('2023-06-30 00:00:00'), 'Holder': 'Vanguard Growth Index Fund', 'pctHeld': 0.0085, 'Sha

In [36]:
print(resp["output"])

The top mutual fund holders of Microsoft stock are:

1. Vanguard Total Stock Market Index Fund - 0.0312% of shares, 231,883,087 shares, and a value of $789,654,664.46
2. Vanguard 500 Index Fund - 0.0238% of shares, 177,034,936 shares, and a value of $602,874,771.05
3. Fidelity 500 Index Fund - 0.0115% of shares, 85,181,943 shares, and a value of $279,192,336.37
4. SPDR S&P 500 ETF Trust - 0.0112% of shares, 83,514,381 shares, and a value of $263,696,658.00
5. iShares Core S&P 500 ETF - 0.0095% of shares, 70,264,910 shares, and a value of $221,861,453.32

Note: The percentages and values are based on the data provided and may not reflect the current holdings.


In [37]:
resp = finance_agent_executor.invoke({"messages": [HumanMessage(content="Top institutional holders of microsoft stock.")]})



> Entering new AgentExecutor chain...

Invoking: `summary_of_institutional_holders` with `{'ticker': 'MSFT'}`


[{'Date Reported': Timestamp('2023-06-30 00:00:00'), 'Holder': 'Vanguard Group Inc', 'pctHeld': 0.0879, 'Shares': 653247546, 'Value': 222456919314}, {'Date Reported': Timestamp('2023-06-30 00:00:00'), 'Holder': 'Blackrock Inc.', 'pctHeld': 0.0722, 'Shares': 536245027, 'Value': 182612881494}, {'Date Reported': Timestamp('2023-06-30 00:00:00'), 'Holder': 'State Street Corporation', 'pctHeld': 0.0395, 'Shares': 293614078, 'Value': 99987338122}, {'Date Reported': Timestamp('2023-06-30 00:00:00'), 'Holder': 'FMR, LLC', 'pctHeld': 0.028299998, 'Shares': 210111250, 'Value': 71551285075}, {'Date Reported': Timestamp('2023-06-30 00:00:00'), 'Holder': 'Price (T.Rowe) Associates Inc', 'pctHeld': 0.0215, 'Shares': 159904219, 'Value': 54453782738}, {'Date Reported': Timestamp('2023-06-30 00:00:00'), 'Holder': 'Geode Capital Management, LLC', 'pctHeld': 0.0199, 'Shares': 148129470, 'Valu

In [38]:
print(resp["output"])

Based on the output from the tool call, the top institutional holders of Microsoft stock are:

1. Vanguard Group Inc - 8.79% of shares, 653,247,546 shares, and a value of $222,456,919,314
2. Blackrock Inc. - 7.22% of shares, 536,245,027 shares, and a value of $182,612,881,494
3. State Street Corporation - 3.95% of shares, 293,614,078 shares, and a value of $99,987,338,122
4. FMR, LLC - 2.83% of shares, 210,111,250 shares, and a value of $71,551,285,075
5. Price (T.Rowe) Associates Inc - 2.15% of shares, 159,904,219 shares, and a value of $54,453,782,738

Note that the percentages and values are based on the data provided and may not reflect the current holdings or values.


In [44]:
resp = finance_agent_executor.invoke({"messages": [HumanMessage(content="Which firms recently changed grade of apple stock? Also, return rating changes.")]})



> Entering new AgentExecutor chain...

Invoking: `stock_grade_updrages_downgrades` with `{'ticker': 'AAPL'}`


[{'Firm': 'Rosenblatt', 'ToGrade': 'Buy', 'FromGrade': 'Neutral', 'Action': 'up'}, {'Firm': 'Erste Group', 'ToGrade': 'Buy', 'FromGrade': 'Hold', 'Action': 'up'}, {'Firm': 'DA Davidson', 'ToGrade': 'Buy', 'FromGrade': 'Neutral', 'Action': 'up'}, {'Firm': 'Itau BBA', 'ToGrade': 'Market Perform', 'FromGrade': 'Underperform', 'Action': 'up'}, {'Firm': 'Bernstein', 'ToGrade': 'Outperform', 'FromGrade': 'Market Perform', 'Action': 'up'}, {'Firm': 'DZ Bank', 'ToGrade': 'Hold', 'FromGrade': 'Buy', 'Action': 'down'}, {'Firm': 'B of A Securities', 'ToGrade': 'Buy', 'FromGrade': 'Neutral', 'Action': 'up'}, {'Firm': 'Redburn Atlantic', 'ToGrade': 'Neutral', 'FromGrade': 'Buy', 'Action': 'down'}, {'Firm': 'Piper Sandler', 'ToGrade': 'Neutral', 'FromGrade': 'Overweight', 'Action': 'down'}, {'Firm': 'Barclays', 'ToGrade': 'Underweight', 'FromGrade': 'Equal-Weight', 'Action': 'down'}]
Invo

In [45]:
print(resp["output"])

Firms that recently changed the grade of Apple stock are:

* Rosenblatt: Upgraded from Neutral to Buy
* Erste Group: Upgraded from Hold to Buy
* DA Davidson: Upgraded from Neutral to Buy
* Itau BBA: Upgraded from Underperform to Market Perform
* Bernstein: Upgraded from Market Perform to Outperform
* DZ Bank: Downgraded from Buy to Hold
* Redburn Atlantic: Downgraded from Buy to Neutral
* Piper Sandler: Downgraded from Overweight to Neutral
* Barclays: Downgraded from Equal-Weight to Underweight


In [41]:
resp = finance_agent_executor.invoke({"messages": [HumanMessage(content="Can you share latest news articles links discussing apple stocks?")]})



> Entering new AgentExecutor chain...

Invoking: `stock_news` with `{'ticker': 'AAPL'}`


[{'uuid': '21d480c4-1a74-3fd2-a7ec-696af853b5e1', 'title': 'Apple Watch to have larger face size in Series 10 models', 'publisher': 'Fox Business', 'link': 'https://finance.yahoo.com/news/apple-watch-larger-face-size-213340551.html', 'providerPublishTime': 1720474420, 'type': 'STORY', 'thumbnail': {'resolutions': [{'url': 'https://s.yimg.com/uu/api/res/1.2/uqsXuxyiXwhVi0xC2XqfRw--~B/aD03MjA7dz0xMjgwO2FwcGlkPXl0YWNoeW9u/https://media.zenfs.com/en/fox_business_text_367/db6ff7ab8aa7f34d385be406f1a2d7b8', 'width': 1280, 'height': 720, 'tag': 'original'}, {'url': 'https://s.yimg.com/uu/api/res/1.2/rquFNwGfHET2kOqmsCYQEw--~B/Zmk9ZmlsbDtoPTE0MDtweW9mZj0wO3c9MTQwO2FwcGlkPXl0YWNoeW9u/https://media.zenfs.com/en/fox_business_text_367/db6ff7ab8aa7f34d385be406f1a2d7b8', 'width': 140, 'height': 140, 'tag': '140x140'}]}, 'relatedTickers': ['AAPL']}, {'uuid': '32556746-6b5b-3e2d-a1cc-8ce92e319db3', 'title': 'Th

In [42]:
print(resp["output"])

Here are the latest news articles discussing Apple stocks:

1. "Apple Watch to have larger face size in Series 10 models" - Fox Business
https://finance.yahoo.com/news/apple-watch-larger-face-size-213340551.html

2. "The Best Costco Finds This Week" - Motley Fool
https://finance.yahoo.com/m/32556746-6b5b-3e2d-a1cc-8ce92e319db3/the-best-costco-finds-this.html

3. "Commercial real estate, Good Buy or Goodbye: Market Domination" - Yahoo Finance Video
https://finance.yahoo.com/video/commercial-real-estate-good-buy-212002468.html

4. "Microsoft to offer Apple devices to employees in China, cites absence of Android services" - Reuters
https://finance.yahoo.com/news/microsoft-offer-apple-devices-employees-203821062.html

5. "Top Wall Street strategist explains why he's abandoning an S&P 500 target" - Yahoo Finance
https://finance.yahoo.com/news/top-wall-street-strategist-explains-why-hes-abandoning-an-sp-500-target-203413713.html

6. "Apple removes VPN apps at request of Russian authorities, 

## Summary

In this video, I explained how to create **finance agent** using **LangChain**. We used open source LLM **llama-3** for our purpose.